# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [ ]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 94kB/s 
     |████████████████████████████████| 450kB 32.2MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 3.8MB 23.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=3c116d5d37d17963892dae7d5fc71a3fe76d5d896922b9db32f8ee9e79ae0e54
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstal

## Get Required Files from Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#Set your project path 
project_path =  '/content/drive/MyDrive/Sarcasm Detection/Data/'

In [3]:
cd drive/MyDrive/Sarcasm Detection/Data/

/content/drive/.shortcut-targets-by-id/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk/Sarcasm Detection/Data


In [4]:
ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt   Sarcasm_Headlines_Dataset.json


#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [62]:
import pandas as pd
sarcasm_head = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)
#sarcasm_head = pd.read_json(project_path + 'Sarcasm_Headlines_Dataset.json',orient='split')
#, orient='records')

In [63]:
sarcasm_head.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [64]:
sarcasm_head.groupby(['is_sarcastic']).agg(['count'])

,article_link,headline
,count,count
is_sarcastic,,
0,14985,14985
1,11724,11724


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [65]:
sarcasm_head = sarcasm_head.drop(['article_link'], axis=1)

In [66]:
sarcasm_head.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [67]:
print(len(sarcasm_head["headline"][0]))

78


In [68]:
import numpy as np

In [69]:
#print()
en = np.zeros(sarcasm_head.headline.count())##, dtype=int)
#en
for i in range(0,sarcasm_head.headline.count()):
  en[i] = len(sarcasm_head["headline"][i])

In [70]:
print(en)

[78. 84. 79. ... 21. 60. 33.]


In [71]:
print("The maximum length is ", en.max(), "and the min length is", en.min())

The maximum length is  254.0 and the min length is 7.0


#**## Modelling**

## Import required modules required for modelling.

In [72]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [73]:
max_features = 10000
maxlen = int(en.max())
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [74]:
tokenizer = Tokenizer(max_features)
tokenizer.fit_on_texts(sarcasm_head['headline'])

In [75]:
sarcasm_head.dtypes

headline        object
is_sarcastic     int64
dtype: object

# Define X and y for your model.

In [76]:
X = tokenizer.texts_to_sequences(sarcasm_head['headline'])
##print(X)
##"""
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(sarcasm_head['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])
##"""

Number of Samples: 26709
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0  

## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [77]:
print("Total Vocab size is ", len(tokenizer.word_index))
num_words = len(tokenizer.word_index)
##tokenizer.word_index()

Total Vocab size is  29656


#**## Word Embedding**

## Get Glove Word Embeddings

### Do not run again - unziping files

In [37]:
glove_file = project_path + "glove.6B.zip"

In [30]:
cd /content/drive/MyDrive/Sarcasm/

/content/drive/MyDrive/Sarcasm


In [38]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [78]:
EMBEDDING_FILE = './glove.6B.200d.txt' ### 200 dimensions is picked because embedding_size is 200. If embedding_size was 50 we would have chosed glove embedding with 50d i.e. the file name glove.6B.50d.txt

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [79]:
embedding_matrix = np.zeros((num_words+1, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [111]:
### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))

model = Sequential()
model.add(Embedding(num_words+1, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
#model.add(Dense(256))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(64))
#model.add(Dropout(0.2))
model.add(Dense(32))
#model.add(Dropout(0.2))
model.add(Dense(8))
model.add(Dense(1,activation='sigmoid'))

In [112]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, None, 200)         5931400   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, None, 256)         336896    
_________________________________________________________________
dense_56 (Dense)             (None, None, 128)         32896     
_________________________________________________________________
dropout_16 (Dropout)         (None, None, 128)         0         
_________________________________________________________________
dense_57 (Dense)             (None, None, 64)          8256      
_________________________________________________________________
dense_58 (Dense)             (None, None, 32)          2080      
_________________________________________________________________
dense_59 (Dense)             (None, None, 8)         

In [115]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [116]:
batch_size = 100
epochs = 5
model.fit(X, y, epochs=5, batch_size=100, validation_split=0.2)
## Add your code here ##

Epoch 1/5
214/214 [==============================] - 28s 117ms/step - loss: 0.4354 - accuracy: 0.7958 - val_loss: 0.3545 - val_accuracy: 0.8519
Epoch 2/5
214/214 [==============================] - 24s 111ms/step - loss: 0.1786 - accuracy: 0.9338 - val_loss: 0.3890 - val_accuracy: 0.8526
Epoch 3/5
214/214 [==============================] - 24s 110ms/step - loss: 0.1179 - accuracy: 0.9574 - val_loss: 0.4123 - val_accuracy: 0.8457
Epoch 4/5
214/214 [==============================] - 24s 111ms/step - loss: 0.0808 - accuracy: 0.9730 - val_loss: 0.5571 - val_accuracy: 0.8440
Epoch 5/5
214/214 [==============================] - 24s 110ms/step - loss: 0.0582 - accuracy: 0.9806 - val_loss: 0.5870 - val_accuracy: 0.8435


# Testing

In [96]:
y_predict = model.predict(X)

In [102]:
print(sarcasm_head.headline[124])

explosion fells building outside paris, killing at least 2


In [103]:
X.shape

(26709, 254)

In [104]:
y.shape

(26709,)

In [106]:
y_predict.shape
print(y_predict[124])
#print("Predicted is ", y_predict[123],"and Actual is", y[123])

[[0.1615266 ]
 [0.16547391]
 [0.1682866 ]
 [0.17003538]
 [0.17086448]
 [0.17098483]
 [0.1706465 ]
 [0.17009865]
 [0.16955072]
 [0.16914815]
 [0.16896477]
 [0.16901089]
 [0.16924879]
 [0.16961092]
 [0.17001893]
 [0.17039827]
 [0.17069082]
 [0.17086214]
 [0.17090416]
 [0.1708327 ]
 [0.17068118]
 [0.17049095]
 [0.17030303]
 [0.17015013]
 [0.17005283]
 [0.17001799]
 [0.17004001]
 [0.17010489]
 [0.17019324]
 [0.17028552]
 [0.17036502]
 [0.17041981]
 [0.17044497]
 [0.17044118]
 [0.17041446]
 [0.17037366]
 [0.17032847]
 [0.17028761]
 [0.17025745]
 [0.17024139]
 [0.17023967]
 [0.17025   ]
 [0.17026834]
 [0.17029007]
 [0.17031074]
 [0.17032695]
 [0.17033659]
 [0.17033921]
 [0.17033565]
 [0.17032759]
 [0.17031726]
 [0.17030698]
 [0.1702984 ]
 [0.17029282]
 [0.17029066]
 [0.17029169]
 [0.17029513]
 [0.17029989]
 [0.17030501]
 [0.17030944]
 [0.17031248]
 [0.17031401]
 [0.17031391]
 [0.17031254]
 [0.17031038]
 [0.17030784]
 [0.17030564]
 [0.1703039 ]
 [0.17030296]
 [0.17030287]
 [0.17030334]
 [0.17

In [117]:
print(sarcasm_head.headline[123])

dad recommends hotel 10 miles away from city you're visiting


In [118]:
print("Predicted is ", y_predict[123],"and Actual is", y[123])

Predicted is  [[0.8499248 ]
 [0.8535693 ]
 [0.8560799 ]
 [0.8576062 ]
 [0.85832083]
 [0.858424  ]
 [0.85813344]
 [0.8576609 ]
 [0.8571859 ]
 [0.85683507]
 [0.8566749 ]
 [0.85671526]
 [0.85692286]
 [0.8572381 ]
 [0.857592  ]
 [0.85791963]
 [0.8581715 ]
 [0.8583187 ]
 [0.8583548 ]
 [0.8582934 ]
 [0.8581633 ]
 [0.8579996 ]
 [0.85783744]
 [0.85770535]
 [0.8576214 ]
 [0.8575911 ]
 [0.8576102 ]
 [0.85766625]
 [0.8577427 ]
 [0.8578225 ]
 [0.85789096]
 [0.85793823]
 [0.85795987]
 [0.8579567 ]
 [0.8579336 ]
 [0.8578984 ]
 [0.8578594 ]
 [0.85782415]
 [0.85779816]
 [0.8577843 ]
 [0.8577828 ]
 [0.85779166]
 [0.8578075 ]
 [0.8578263 ]
 [0.85784423]
 [0.85785806]
 [0.85786647]
 [0.8578688 ]
 [0.85786563]
 [0.85785866]
 [0.85784984]
 [0.8578409 ]
 [0.8578335 ]
 [0.8578287 ]
 [0.8578268 ]
 [0.8578277 ]
 [0.8578307 ]
 [0.85783476]
 [0.8578391 ]
 [0.857843  ]
 [0.8578457 ]
 [0.85784703]
 [0.8578469 ]
 [0.8578457 ]
 [0.8578439 ]
 [0.8578417 ]
 [0.8578398 ]
 [0.8578382 ]
 [0.8578374 ]
 [0.85783726]
 [0.85

### I wonder why y predict is a 254x1 for a single headline. Shouldn't it be just 1?